In [1]:
# Analyze results from compare_cl_vs_vanilla

In [1]:
import os
import pandas as pd

In [75]:
base = '/home/cgn/amazon_reviews/'
results = []
for seed in range(5):
    for trainsize in [500000, 1000000]:
        for epochs in [5, 20, 50]:
            if trainsize == 500000 and epochs == 50:
                continue
            fn = 'out_seed_{}_trainsize_{}_epochs_{}.log'.format(
                    seed, trainsize, epochs)
#             print(fn)
            with open(base + fn, 'r') as f:
                result = f.readlines()
                lod = eval(result[-5:-4][0].strip())
                settings = {'seed': seed,
                            'trainsize': trainsize,
                            'epochs': epochs}
                [d.update(settings) for d in lod]
                results += lod

In [76]:
data = pd.DataFrame(results)
data = data[data['method'] != 'cl_intersection_all_methods']
data.drop('train_size', axis=1, inplace=True)
data['trainsize'] = (data['trainsize'] / 1000).round().astype(int).astype(str) + 'K'

In [83]:
cols = ['Top-1 Acc', 'Pruned']
df = data.groupby(['test_split', 'trainsize', 'epochs', 'method']).agg(['mean', 'std'])
# df.columns = [' '.join(col).strip() for col in df.columns.values]
df.drop([('seed', 'mean'), ('seed', 'std')], axis=1, inplace=True)
# Formatting
df[('data_removed', 'mean')] = (df[('data_removed', 'mean')] / 1000).round().astype(int).astype(str) + 'K'
# df[('data_removed', 'std')] = df[('data_removed', 'std')].round().astype(int)
del df[('data_removed', 'std')]
df[('acc', 'mean')] = (df[('acc', 'mean')] * 100).round(1)
df[('acc', 'std')] = (df[('acc', 'std')] * 100).round(2)
df.columns = [' '.join(col).strip() for col in df.columns.values]
df['acc'] = df['acc mean'].astype(str) + '±' + df['acc std'].astype(str)
del df['acc mean']
del df['acc std']
df = df[['acc', 'data_removed mean']]
df.columns = cols
# pd.concat([z for i, z in df.reset_index().groupby(['test_split', 'trainsize'])])
sdfs = []
for key, sdf in df.reset_index(level=[1,2]).groupby(['trainsize', 'epochs']):
    z = sdf[cols]
    label1 = r'$N = {}$'.format(key[0])
    label2 = 'epochs = {}'.format(key[1])
    z.columns = pd.MultiIndex.from_product([[label1], [label2], z.columns])
    sdfs.append(z)
paper_df = pd.concat(sdfs, axis=1)
for c in [('$N = 1000K$',  'epochs = 5',    'Pruned'),
          ('$N = 1000K$', 'epochs = 20',    'Pruned'),
          ('$N = 500K$',  'epochs = 5',     'Pruned'),]:
    del paper_df[c]
paper_df

$N = 1000K$                                $N = 500K$  \
                    epochs = 5 epochs = 20 epochs = 50        epochs = 5   
                     Top-1 Acc   Top-1 Acc   Top-1 Acc Pruned  Top-1 Acc   
test_split method                                                          
10         argmax    85.2±0.06   89.2±0.02   90.0±0.02   291K  86.6±0.03   
           both      86.3±0.04   89.8±0.01   90.2±0.01   250K  87.5±0.05   
           cj_only   86.4±0.01   89.8±0.02   90.1±0.02   246K  87.5±0.02   
           pbc       86.2±0.03   89.7±0.01   90.2±0.01   257K  87.4±0.03   
           pbnr      86.2±0.07   89.7±0.01   90.2±0.01   257K  87.4±0.05   
           vanilla   83.9±0.11   86.3±0.06   84.4±0.04     0K  82.7±0.07   
11         argmax    85.3±0.05   89.3±0.01    90.0±0.0   294K  86.6±0.04   
           both      86.4±0.06   89.8±0.01   90.2±0.01   252K  87.5±0.04   
           cj_only   86.3±0.05   89.8±0.01   90.1±0.02   249K  87.5±0.03   
           pbc       86.2±0.03   89.8±0.01    90.3±0.0   260K  87.4±0.03   
           pbnr      86.2±0.06   89.8±0.01   90.2±0.02   260K  87.4±0.05   
           vanilla   83.9±0.08   86.3±0.05   84.4±0.12     0K  82.7±0.04   

                                       
                   epochs = 20         
                     Top-1 Acc Pruned  
test_split method                      
10         argmax    86.6±0.03   259K  
           both      87.5±0.03   244K  
           cj_only   87.5±0.02   243K  
           pbc       87.4±0.03   247K  
           pbnr      87.4±0.05   247K  
           vanilla   82.8±0.07     0K  
11         argmax    86.6±0.06   261K  
           both      87.5±0.03   247K  
           cj_only   87.5±0.02   246K  
           pbc       87.4±0.05   250K  
           pbnr      87.4±0.03   249K  
           vanilla   82.7±0.09     0K

In [84]:
method_name_map = {
	'argmax': r'CL: $\bm{C}_{\text{confusion}}$',
	'pbc': 'CL: PBC',
	'cj\_only': r'CL: $\cj$',
	'both': 'CL: C+NR',
	'pbnr': 'CL: PBNR',
	'vanilla': 'Baseline',
}

In [81]:
tex = paper_df.to_latex().replace('\\$', '$').replace('±', '$\pm$')
tex = ' '.join([method_name_map. get(i, i) for i in tex.split(' ')])
print(tex)

\begin{tabular}{lllllllll}
\toprule
   &         & \multicolumn{4}{l}{$N = 1000K$} & \multicolumn{3}{l}{$N = 500K$} \\
   &         &  epochs = 5 & epochs = 20 & \multicolumn{2}{l}{epochs = 50} & epochs = 5 & \multicolumn{2}{l}{epochs = 20} \\
   &         &   Top-1 Acc &   Top-1 Acc &   Top-1 Acc & Pruned &  Top-1 Acc &   Top-1 Acc & Pruned \\
test\_split & method &             &             &             &        &            &             &        \\
\midrule
10 & CL: $\bm{C}_{\text{confusion}}$ &   85.2$\pm$0.06 &   89.2$\pm$0.02 &   90.0$\pm$0.02 &   291K &  86.6$\pm$0.03 &   86.6$\pm$0.03 &   259K \\
   & CL: C+NR &   86.3$\pm$0.04 &   89.8$\pm$0.01 &   90.2$\pm$0.01 &   250K &  87.5$\pm$0.05 &   87.5$\pm$0.03 &   244K \\
   & CL: $\cj$ &   86.4$\pm$0.01 &   89.8$\pm$0.02 &   90.1$\pm$0.02 &   246K &  87.5$\pm$0.02 &   87.5$\pm$0.02 &   243K \\
   & CL: PBC &   86.2$\pm$0.03 &   89.7$\pm$0.01 &   90.2$\pm$0.01 &   257K &  87.4$\pm$0.03 &   87.4$\pm$0.03 &   247K \\
   & CL: PBNR 